In [164]:
from pyne import data
import numpy as np
import scipy.constants as const
import tabulate

Define useful constants

$ I_{Al} $ from [Bogaardt](http://dspace.library.uu.nl/bitstream/handle/1874/24230/bogaardt_koudijs_51_average_excitation_potentials_air_aluminium.pdf?sequence=1)

$ \rho = 2.70 g/cm^3 $ from [Wikipedia entry for Alumnium](https://en.wikipedia.org/wiki/Aluminium)

In [165]:
m_e = const.value('electron mass energy equivalent in MeV')
I_Al = 163 * 10**-6 # eV to MeV
r_0 = const.value('classical electron radius') * 100 # m to cm
z = 1
Z = 13
N_A = const.value('Avogadro constant')
M_m = data.atomic_mass('Al')
rho = 2.70
pi = const.pi
m_p = const.value('proton mass energy equivalent in MeV')

  Useful conversion functions
 

In [166]:
def beta(gamma):
    g = gamma*gamma
    b = (1 - g ** -1) ** 0.5
    # return ((g - 1.0) / g) ** 0.5
    return b
def gamma(T, m):
    return (T / m) + 1.0
def beta_T(T, m):
    denominator = (T + m) ** 2.0
    numerator = T * (T + 2.0 * m)
    return (numerator / denominator) ** 0.5
def percent_error(truth, model):
    return abs(truth - model) / truth

Input energies

In [167]:
T = [10, 100, 500]

In [168]:
def S_classical(T):
    # We're dealing with an incident proton now
    m = m_p
    
    # Get our incident particle energy in terms of Beta
    b = beta_T(T, m)
    
    first_part = 4 * pi * r_0**2 * m_e
    incident_particle_part = z**2 / b**2
    medium_part = Z * N_A * rho / M_m
    log_term = 2 * m_e * gamma(T, m)**2 * b**2 / I_Al
    last_part = np.log(log_term)
    
    return first_part * incident_particle_part * medium_part * last_part
def S_relativistic(T):
    # We're dealing with an incident proton now
    m = m_p
    
    # Get our incident particle energy in terms of Beta
    b = beta_T(T, m)
    
    first_part = 4 * pi * r_0**2 * m_e
    incident_particle_part = z**2 / b**2
    medium_part = Z * N_A * rho / M_m
    log_term = 2 * m_e * gamma(T, m)**2 * b**2 / I_Al
    last_part = np.log(log_term) - b**2
    
    return first_part * incident_particle_part * medium_part * last_part

Classical Results

In [169]:
out = []
headers = ['T', 'Classical', 'Relativistic', 'Percent Error']
for t in T:
    out.append((t, S_classical(t), S_relativistic(t), percent_error(S_relativistic(t), S_classical(t))))
print tabulate.tabulate(out, headers=headers)

  T    Classical    Relativistic    Percent Error
---  -----------  --------------  ---------------
 10     93.3096         92.9102        0.00429954
100     15.7951         15.3956        0.0259471
500      6.28905         5.88958       0.0678267


Problem 4

In [170]:
t = 2.5

From fig 2.11, with $ \frac{T/A}{Z} = \frac{2.5}{13} = 0.19 $, the shell correction is approximately $ \lambda = 0.1 $

In [171]:
def S_relativistic_effective_charge(T):
    # We're dealing with an incident proton now
    m = m_p
    
    # Get our incident particle energy in terms of Beta
    b = beta_T(T, m)
    
    # From Anderson fig 2.11
    shell_correction = 0.1
    
    first_part = 4 * pi * r_0**2 * m_e
    incident_particle_part = z**2 / b**2
    medium_part = Z * N_A * rho / M_m
    log_term = 2 * m_e * gamma(T, m)**2 * b**2 / I_Al
    last_part = np.log(log_term) - b**2 - shell_correction
    
    return first_part * incident_particle_part * medium_part * last_part

No corrections:

In [172]:
S_classical(t)

264.18888757811038

With corrections:

In [173]:
S_relativistic_effective_charge(t)

256.26320216888456